In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from researchlib.single_import import *

In [3]:
model = builder([
    Reshape((-1, 784)),
    BinarizeLinear(784, 512),
    nn.BatchNorm1d(512),
    nn.Hardtanh(),
    BinarizeLinear(512, 256),
    nn.BatchNorm1d(256),
    nn.Hardtanh(),
    BinarizeLinear(256, 128),
    nn.BatchNorm1d(128),
    nn.Hardtanh(),
    nn.Linear(128, 10)  
])

In [4]:
train_loader = FromPublic('mnist', 'train', batch_size=128, normalize=True, pin_memory=True, num_workers=4)
test_loader = FromPublic('mnist', 'test', batch_size=128, normalize=True, pin_memory=True, num_workers=4)

In [5]:
runner = Runner(model, train_loader, test_loader, 'adam', 'focal')

In [7]:
for _ in range(3): runner.fit_onecycle(1e-1, mixup_alpha=0.4, callbacks=[Binarized()])

0.7499729264259338
0.9477

Test set: Average loss: 0.1546



0.778111104520162
0.9522

Test set: Average loss: 0.1540



0.7839438592910767
0.9495

Test set: Average loss: 0.1789

